In [1]:
import os
from ml4floods.data import utils
import geopandas as gpd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ml4cc-general-access_request_pays.json"

## Step 0: load the manually drawn polygons

Polygons could be given or manually drawn using http://geojson.io/, they could also be drawn using QGIS or an automatic script. If you draw them keep them small (250 x 250 km²)

In [3]:
data = gpd.read_file("/home/gonzalo/aoi_valencia.shp")
data

,name,geometry
0,VALENCIA,"POLYGON ((-0.63004 39.57777, -0.24809 39.55850..."
1,CULLERA,"POLYGON ((-0.20525 39.39594, -0.13743 39.11262..."
2,CHIVA,"POLYGON ((-0.50510 39.65064, -0.48011 39.33108..."
3,XATIVA,"POLYGON ((-0.71928 39.19532, -0.32662 39.17180..."
4,CORTESDEPALLAS,"POLYGON ((-1.02448 39.34144, -0.78175 39.34765..."
5,SAGUNT,"POLYGON ((-0.56177 39.78998, -0.09415 39.78861..."
6,GANDIA,"POLYGON ((-0.11423 39.15588, 0.03748 38.88132,..."


## Step 1 Save this in GCP bucket as GeoJSON

In [4]:
utils.write_geojson_to_gcp("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/VAL001/aois.json", data)

## Step 2: Add the pickle files to the GCP location

In [ ]:
import datetime

for row in data.itertuples():
    ems_code = "VAL001"
    aoi_code = row.name
    satellite_date = datetime.datetime(2022, 5, 3)
    data_save = {'satellite date': satellite_date,
                 'area_of_interest_polygon': row.geometry,
                 'ems_code': ems_code,
                 'aoi_code': aoi_code,
                 'date_ems_code': satellite_date}
    gs_path = f"gs://ml4cc_data_lake/0_DEV/1_Staging/operational/EMSR570/{aoi_code}/flood_meta/{satellite_date.strftime('%Y-%m-%d')}.pickle"
    utils.write_pickle_to_gcp(gs_path, data_save)